In [137]:
import re
import pandas as pd
import numpy as np
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import spacy


In [138]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('snowball_data')
spacy.cli.download('es_core_news_sm')
nlp = spacy.load("es_core_news_sm")
stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package stopwords to /Users/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/david/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     /Users/david/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 23.7 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [139]:
def remove_spanish_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [140]:
def stem_spanish_text(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [141]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [142]:
def extract_texts(filename):
    with open(filename, 'r') as f:
        input_text = f.read()
    matches = re.finditer(r"-={Concordancia}=-\n(.*?)\n-={Referencia bibliográfica}=-", input_text, re.DOTALL)
    extracted_texts = []
    for match in matches:
        extracted_text = match.group(1).strip()
        extracted_texts.append(extracted_text)
    return extracted_texts

In [143]:
all_texts = []
all_texts = [*all_texts, *extract_texts('economia.txt')]
all_texts = [*all_texts, *extract_texts('politica.txt')]
all_texts = [*all_texts, *extract_texts('medicina.txt')]

In [144]:
print(all_texts[0])

todos sus elementos, así los llamados "espirituales" como los que se dicen "materiales", pues unos y otros, pese a esa convencional dicotomía, no son sino hechos igualmente humanos, interdependientes e integrantes de la plenitud de esa cultura. No solamente el folklore, sino las artes literarias y plásticas, así como la economía, el derecho, la magia y todos los demás hechos humanos de cualquier pueblo o época son igualmente objeto de la ciencia antropológica, especialmente de la etnografía. El estudio de un canto tiene tanto valor, o más valor, que el de un ángulo facial, las mudanzas de un baile no significan menos que la


In [145]:
df = pd.DataFrame(all_texts, columns=['text'])
display(df)

,text
0,"todos sus elementos, así los llamados ""espirit..."
1,Del consumo económico de tales areítos puede d...
2,No puede negarse que en tales funciones había ...
3,cuando llevara al exceso. Hubiera sido ofender...
4,En todo tiempo ha existido el alarde de riquez...
...,...
2998,. Había 15 veces menos estudiantes de este tip...
2999,La situación nacional de la enseñanza de cienc...
3000,nacional de la enseñanza de ciencias de la sal...
3001,A este respecto pueden mencionarse dos indicad...


In [146]:
df['text'] = df['text'].apply(remove_spanish_stopwords)
df['text'] = df['text'].apply(lemmatize_text)

In [147]:
display(df)

,text
0,"elemento , así llamado "" espiritual "" decir "" ..."
1,consumo económico tal areíto poder decir él an...
2,poder negar él tal función considerable derroc...
3,llevar exceso . ser ofender canon buen gusto s...
4,"tiempo existir alarde riqueza trascendente "" f..."
...,...
2998,. 15 vez menos estudiante tipo medicina número...
2999,"situación nacional enseñanzar ciencia salud , ..."
3000,"nacional enseñanzar ciencia salud , punto vist..."
3001,respecto poder mencionar él dos indicador ilus...


In [148]:
vectorizer_tfidf = TfidfVectorizer()
vectors_tfidf = vectorizer_tfidf.fit_transform(df['text']).todense()
vocab = np.array(vectorizer_tfidf.get_feature_names_out())
print(vectors_tfidf.shape)
print(len(vocab))
print(vectors_tfidf)

(3003, 16764)
16764
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [149]:
num_top_words = 1
d = 3

def show_topics(vocab, H):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in H])
    return [' '.join(t) for t in topic_words]

In [150]:
nmf = decomposition.NMF(n_components=d)

W = nmf.fit_transform(np.asarray(vectors_tfidf))
H = nmf.components_

In [151]:
print(W.shape)
print(H.shape)
print(H)

(3003, 3)
(3, 16764)
[[0.00058605 0.00100175 0.00128326 ... 0.07997246 0.01176431 0.00215774]
 [0.00172045 0.         0.         ... 0.00363879 0.00187594 0.00048085]
 [0.         0.         0.         ... 0.03153298 0.         0.        ]]


In [152]:
show_topics(vocab, H)

['medicina', 'economía', 'político']